## Atomic Trajectory Analysis

### Step 1: Importing Libraries

In [ ]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PSF, DCD, GRO, XTC
from matplotlib import pyplot as plt
from MDAnalysis.analysis.distances import distance_array
import numpy as np
import pandas as pd
from MDAnalysis.analysis import rms
from MDAnalysis.analysis.rms import RMSF
from MDAnalysis.analysis.align import alignto
import MDAnalysis.transformations as trans

### Step 2: Loading Trajectory

In [ ]:
u = mda.Universe("box.pdb", "aligned_centered_traj.xtc")

In [ ]:
last_time = u.trajectory[-1].time  
print(f"The simulation ran until {last_time} ps.")

### Step 3: Select Protein Atoms

In [3]:
protein = u.select_atoms('protein') 

In [ ]:
print(u)
print(len(u.trajectory))

In [ ]:
print(u.atoms.segids)

In [ ]:
print(u.residues) 
print(u.residues.resnames)

### Step 4: Select N-terminal and C-terminal Atoms

In [ ]:
nterm = u.select_atoms('segid A and name N')[0]
cterm = u.select_atoms('segid A and name C')[-1]

### Step 5: Analyze End-to-End Distance and Radius of Gyration Over Time

In [ ]:
# Initialize lists to store analysis results
frames = []
distances = []
rgyr_values = []

# Loop through each frame of the trajectory
for ts in u.trajectory:
    # Compute the end-to-end vector between C-terminal and N-terminal
    r = cterm.position - nterm.position
    d = np.linalg.norm(r)  # Calculate the magnitude of the vector (distance)

    # Compute the radius of gyration of the protein
    rgyr = protein.radius_of_gyration()

    # Store results
    frames.append(ts.frame)
    distances.append(d)
    rgyr_values.append(rgyr)

# Get timestep in picoseconds
dt_ps = u.trajectory.dt  
# Convert frame numbers to time in nanoseconds
times_ns = [frame * dt_ps / 1000 for frame in frames]  

# Plot End-to-End Distance 
plt.figure(figsize=(10, 5))
plt.plot(times_ns, distances, label="End-to-End Distance (Å)", color="b")
plt.xlabel("Time (ns)")
plt.ylabel("Distance (Å)")
plt.title("End-to-End Distance Over Time")
plt.legend()
plt.grid()

# Plot Radius of Gyration
plt.figure(figsize=(10, 5))
plt.plot(times_ns, rgyr_values, label="Radius of Gyration (Å)", color="r")
plt.xlabel("Time (ns)")
plt.ylabel("Radius of Gyration (Å)")
plt.title("Radius of Gyration Over Time")
plt.legend()
plt.grid()
plt.show()


### Step 6: RMSD Analysis — First vs Last and Time Evolution

In [ ]:
# Set the trajectory to the first frame to get reference coordinates
u.trajectory[0]
ref_coords = protein.positions.copy()  # Copy protein atom positions as reference

# Initialize lists to store RMSD values and corresponding times
rmsd_values = []
times = []

# Loop through all frames in the trajectory
for ts in u.trajectory:
    # Calculate RMSD between current frame and reference frame, with alignment (superposition)
    rmsd_val = rms.rmsd(protein.positions, ref_coords, superposition=True)
    rmsd_values.append(rmsd_val)           # Store RMSD value
    times.append(ts.time / 1000)           

# Go to the last frame of the trajectory
u.trajectory[-1]
last_coords = protein.positions.copy()    # Copy protein positions at last frame

# Calculate RMSD between last frame and reference (first) frame, with alignment
rmsd_first_last = rms.rmsd(last_coords, ref_coords, superposition=True)
print(f"RMSD between first and last frame: {rmsd_first_last:.3f} Å")  # Print result

# Plot RMSD over time 
plt.figure(figsize=(10, 5))
plt.plot(times, rmsd_values, color='blue', label="RMSD to First Frame")
plt.xlabel("Time (ns)")     
plt.ylabel("RMSD (Å)")      
plt.title("Protein RMSD Over Time")
plt.legend()
plt.grid()
plt.show()


### Step 7: RMSF Analysis

In [ ]:
# Calculate RMSF for protein atoms over the trajectory
rmsf = RMSF(protein.select_atoms("protein")).run()

# Plot RMSF values against residue index
plt.plot(rmsf.rmsf, color='blue', label="RMSF")
plt.title("Root Mean Square Fluctuation (RMSF) of Protein Residues")
plt.xlabel("Residue index")
plt.ylabel("RMSF (Å)")
plt.legend()
plt.grid(True)
plt.show()